In [295]:
import pandas as pd
import os
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, pipeline
import math
import torch
import numpy as np

input_dir = '../data/inputs'
output_dir = '../data/outputs/s0'
results_dir = '../data/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [281]:


female_ratios = pd.read_csv(os.path.join(input_dir, 'female_ratios.csv'))
# for model in  llama3-8b llama3-8b-instruct mistral-7b mistral-7b-instruct llama2-7b llama2-7b-chat
model_strs = ['llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-chat']
model_strs = sorted(model_strs, key=len, reverse=True)

prompt_ids = ['none','low-1','low-2','medium-3','medium-4','high-5','high-6']
prompt_id_mapping = {pid: idx for idx, pid in enumerate(prompt_ids)}

df_list = []
for f in os.listdir(output_dir):
    if "conv" not in f and "gender" not in f:
    # if 'conversation.csv' in f:
        
        df = pd.read_csv(os.path.join(output_dir, f))
    # if 'conversation.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'plain.csv'), index=False)
    


        

        

In [282]:
df_list = []
for f in os.listdir(output_dir):
    if 'conversation.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'conversation.csv'), index=False)
    

In [293]:
df_list = []
for f in os.listdir(output_dir):
    if 'genderquestion.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])


        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'genderquestion.csv'), index=False)

In [284]:
df_list = []
for f in os.listdir(output_dir):
    if 'genderquestion_conv.csv' in f:
        df = pd.read_csv(os.path.join(output_dir, f))
        df = pd.merge(df,female_ratios,on='job')
        df = df.drop(columns=['job','Unnamed: 0'])
         

        df['female_dominated'] = df['female_ratio'] > 50
        # Extract prompt ID from filename
        prompt_id_str = next((pid for pid in prompt_ids if pid in f), 'none')
        prompt_id = prompt_id_mapping[prompt_id_str]
        df['prompt_id'] = prompt_id
        
                # Extract model from filename
        model_str = next((model for model in model_strs if model in f), None)
        df['model'] = model_str
        df['conversation'] = 'conv' in f

        # Remove model name from other column names
        if model_str:
            df = df.rename(columns=lambda x: x.replace(f'{model_str}_', '') if model_str in x else x)

        
        # grouped_df = df.groupby('female_dominated').mean().reset_index()
        numeric_cols = df.select_dtypes(include='number').columns
        grouped_df = df.groupby(['female_dominated', 'model','conversation'])[numeric_cols].mean().reset_index()
    
        if 'male_met-met' in df.columns:
            col_ends = ['met-met', 'friend', 'talk-met']
            # Compute averages for male, female, and diverse columns
            male_cols = ['male_' + end for end in col_ends]
            female_cols = ['female_' + end for end in col_ends]
            diverse_cols = ['diverse_' + end for end in col_ends]

    
            grouped_df['male'] = grouped_df[male_cols].mean(axis=1)
            grouped_df['female'] = grouped_df[female_cols].mean(axis=1)
            grouped_df['diverse'] = grouped_df[diverse_cols].mean(axis=1)
            # make sure the three columns add up to 1
            # grouped_df['total'] = grouped_df['male'] + grouped_df['female'] + grouped_df['diverse']
            # 
            # Optionally, drop the original detailed columns if only averages are needed
            grouped_df = grouped_df.drop(columns=male_cols + female_cols + diverse_cols)

            
        df_list.append(grouped_df)
    # concat all the dataframes
df = pd.concat(df_list)
df.to_csv(os.path.join(results_dir, 'genderquestion_conv.csv'), index=False)

In [294]:
df.model.unique()

array(['llama2-7b', 'mistral-7b', 'llama3-8b-instruct', 'llama3-8b',
       'mistral-7b-instruct', 'llama2-7b-chat'], dtype=object)

In [285]:
df

,female_dominated,model,conversation,male,female,diverse,female_ratio,prompt_id,alpaca-7b_male,alpaca-7b_female,alpaca-7b_diverse
0,False,llama2-7b,True,0.266465,0.712365,0.021160,5.535,6.0,NaN,NaN,NaN
1,True,llama2-7b,True,0.105650,0.873290,0.021050,89.265,6.0,NaN,NaN,NaN
0,False,mistral-7b,True,0.625040,0.360585,0.014365,5.535,3.0,NaN,NaN,NaN
1,True,mistral-7b,True,0.221395,0.738435,0.040150,89.265,3.0,NaN,NaN,NaN
0,False,mistral-7b,True,0.796830,0.201195,0.001975,5.535,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1,True,mistral-7b-instruct,True,0.019800,0.038490,0.941720,89.265,5.0,NaN,NaN,NaN
0,False,mistral-7b,True,0.708945,0.255865,0.035170,5.535,1.0,NaN,NaN,NaN
1,True,mistral-7b,True,0.353300,0.593430,0.053275,89.265,1.0,NaN,NaN,NaN
0,False,llama3-8b-instruct,True,0.010045,0.026320,0.963625,5.535,1.0,NaN,NaN,NaN


In [286]:
df_plain = pd.read_csv(os.path.join(results_dir, 'plain.csv'))
df_conv = pd.read_csv(os.path.join(results_dir, 'conversation.csv'))

# if any column starts with a space, remove it
df_plain.columns = df_plain.columns.str.strip()
df_conv.columns = df_conv.columns.str.strip()

df = pd.concat([df_plain, df_conv])
for model, group_df in df.groupby('model'):
    # Save the DataFrame to a CSV file
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'non_gq.csv'), index=False)

    # Optional: Print a message to indicate the file has been saved
    # print(f"Saved {model} non_gq.csv")

In [287]:
df

,female_dominated,model,conversation,female_ratio,prompt_id,male,female,diverse,alpaca-7b_male_met-met,alpaca-7b_male_friend,alpaca-7b_male_talk-met,alpaca-7b_female_met-met,alpaca-7b_female_friend,alpaca-7b_female_talk-met,alpaca-7b_diverse_met-met,alpaca-7b_diverse_friend,alpaca-7b_diverse_talk-met
0,False,llama3-8b-instruct,False,5.535,0.0,0.898010,0.049103,0.052882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,llama3-8b-instruct,False,89.265,0.0,0.099750,0.857733,0.042522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,mistral-7b,False,5.535,1.0,0.773295,0.193120,0.033583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,mistral-7b,False,89.265,1.0,0.289362,0.670940,0.039702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,llama2-7b,False,5.535,0.0,0.902528,0.083492,0.013972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,True,mistral-7b-instruct,True,89.265,4.0,0.030102,0.417688,0.552205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,False,llama2-7b-chat,True,5.535,2.0,0.551020,0.015597,0.433387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,True,llama2-7b-chat,True,89.265,2.0,0.044227,0.587288,0.368485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,False,llama3-8b,True,5.535,0.0,0.776022,0.125278,0.098695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [288]:
df_gq_plain = pd.read_csv(os.path.join(results_dir, 'genderquestion.csv'))
df_gq_conv = pd.read_csv(os.path.join(results_dir, 'genderquestion_conv.csv'))

# if any column starts with a space, remove it
df_gq_plain.columns = df_gq_plain.columns.str.strip()
df_gq_conv.columns = df_gq_conv.columns.str.strip()

df_gq = pd.concat([df_gq_plain, df_gq_conv])
for model, group_df in df_gq.groupby('model'):
    # Save the DataFrame to a CSV file
    if not os.path.exists(os.path.join(results_dir,model)):
        os.makedirs(os.path.join(results_dir,model))
    group_df.to_csv(os.path.join(results_dir,model, f'genderquestion.csv'), index=False)

In [289]:
df_gq

,female_dominated,model,conversation,male,female,diverse,female_ratio,prompt_id,alpaca-7b_male,alpaca-7b_female,alpaca-7b_diverse
0,False,llama2-7b,False,0.448720,0.498865,0.052400,5.535,5.0,NaN,NaN,NaN
1,True,llama2-7b,False,0.266260,0.688840,0.044890,89.265,5.0,NaN,NaN,NaN
2,False,mistral-7b,False,0.621235,0.269070,0.109715,5.535,4.0,NaN,NaN,NaN
3,True,mistral-7b,False,0.297795,0.634930,0.067265,89.265,4.0,NaN,NaN,NaN
4,False,llama2-7b,False,0.633945,0.358185,0.007860,5.535,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
79,True,mistral-7b-instruct,True,0.019800,0.038490,0.941720,89.265,5.0,NaN,NaN,NaN
80,False,mistral-7b,True,0.708945,0.255865,0.035170,5.535,1.0,NaN,NaN,NaN
81,True,mistral-7b,True,0.353300,0.593430,0.053275,89.265,1.0,NaN,NaN,NaN
82,False,llama3-8b-instruct,True,0.010045,0.026320,0.963625,5.535,1.0,NaN,NaN,NaN


In [296]:
import os
import pandas as pd

# Define the mapping dictionaries
model_str_map = {
    'llama3-8b': 'Llama3 8B',
    'llama3-8b-instruct': 'Llama3 8B (Instruct)',
    'mistral-7b': 'Mistral 7B',
    'mistral-7b-instruct': 'Mistral 7B (Instruct)',
    'llama2-7b': 'Llama2 7B',
    'llama2-7b-chat': 'Llama2 7B (Chat)',
    # 'alpaca-7b': 'Alpaca 7B',
}

prompt_id_map = {
    0: 'None',
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5',
    6: '6'
}

abstraction_levels = {
    0: '',
    1: 'High',
    2: 'High',
    3: 'Med.',
    4: 'Med.',
    5: 'Low',
    6: 'Low'
}

abs_order = ['', 'High', 'Med.', 'Low']

In [302]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results.tex')

with open(output_tex_file, 'w') as f_out:
    f_out.write(r'\onecolumn')
    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')

        for csv_file, (caption, label_suffix) in [('genderquestion.csv', ('Probability of gender expressions for task prompt number one. ``ID\'\' refers to the id of the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'genderquestion')),
                                                  ('non_gq.csv', ('Aggregated results for task prompts two to four. ``ID\'\' refers to the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'nongq'))]:
            
            
            # Load the CSV file into a DataFrame
            df_path = os.path.join(results_dir, model, csv_file)
            df = pd.read_csv(df_path)

            # Add the abstraction level column to the DataFrame
            df['Abs.'] = df['prompt_id'].map(abstraction_levels)

            # Calculate means for each combination of prompt_id, conversation, and female_dominated
            grouped_df = df.groupby(['Abs.', 'model', 'prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()
            model = grouped_df['model'].unique()[0]

            # Drop the 'model' column
            model_df = grouped_df.drop(columns=['model'])
                        # Sort by Abs. and prompt_id to ensure correct order
            model_df['prompt_id'] = model_df['prompt_id'].astype(int)
            model_df['Abs.'] = pd.Categorical(model_df['Abs.'], categories=abs_order, ordered=True)
            model_df = model_df.sort_values(by=['Abs.', 'prompt_id'])

            latex_table = r'''
\begin{table*}[ht!]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){3-8} \cmidrule(lr){9-14}
    & & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){3-5} \cmidrule(lr){6-8} \cmidrule(lr){9-11} \cmidrule(lr){12-14}
        Abs. & ID & M & F & D & M & F & D & M & F & D & M & F & D\\
        \midrule
'''

            # Sort by prompt_id to ensure order from None to 6
            model_df['prompt_id'] = model_df['prompt_id'].astype(int)
            model_df = model_df.sort_values(by=['Abs.','prompt_id'])

            previous_abs_level = None

            for abs_level, abs_group in model_df.groupby('Abs.'):
                if previous_abs_level is not None:
                    latex_table += "        \\midrule\n"
                for prompt_id, prompt_group in abs_group.groupby('prompt_id'):
                    id_label = 'None' if prompt_id == 0 else str(prompt_id)
                    row_prefix = f"        {abs_level if prompt_id in [0,2,4,6] else ''} & {id_label} & "

                    no_dialogue_fd = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == True)]
                    no_dialogue_md = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == False)]
                    dialogue_fd = prompt_group[(prompt_group['conversation'] == True) & (prompt_group['female_dominated'] == True)]
                    dialogue_md = prompt_group[(prompt_group['conversation'] == True) & (prompt_group['female_dominated'] == False)]

                    row = row_prefix

                    if not no_dialogue_fd.empty:
                        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not no_dialogue_md.empty:
                        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not dialogue_fd.empty:
                        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not dialogue_md.empty:
                        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
                    else:
                        row += " & & "

                    row += r"\\"
                    latex_table += row + "\n"

                if abs_level != '':  # Add averages for all except "None"
                    avg_row_prefix = f"         & \\textbf{{Avg}} & "

                    avg_no_dialogue_fd = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_no_dialogue_md = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_dialogue_fd = abs_group[(abs_group['conversation'] == True) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_dialogue_md = abs_group[(abs_group['conversation'] == True) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)

                    avg_row = avg_row_prefix

                    if not avg_no_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_no_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & & "

                    if not avg_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_dialogue_md['diverse']*100:.1f}\\%}} "
                    else:
                        avg_row += " & & "

                    avg_row += r"\\"
                    latex_table += avg_row + "\n"

                previous_abs_level = abs_level

            latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{''' + caption + r'''}
\label{tab:''' + label_suffix + r'''_''' + model_name + r'''}
\end{table*}

'''

            f_out.write(latex_table)
        f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/aggregated_results.tex


/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/2488884905.py:60: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/2488884905.py:60: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/2488884905.py:60: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or

In [308]:
 # Update this to your results directory
models = list(model_str_map.keys())

output_tex_file = os.path.join(results_dir, 'aggregated_results_no_diag.tex')

with open(output_tex_file, 'w') as f_out:
    f_out.write(r'\onecolumn')
    for model in models:
        model_name = model.replace('-', '')
        model_description = model_str_map[model]

        f_out.write(r'\subsection{' + model_description + '}\n')

        for csv_file, (caption, label_suffix) in [('genderquestion.csv', ('Result for explicit bias (task prompt 1). ``ID\'\' refers to the id of the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'genderquestion')),
                                                  ('non_gq.csv', ('Result for explicit bias (average over task prompt 2-4). ``ID\'\' refers to the debiasing prompt, ``Abs.\'\' indicates its degree of abstraction.', 'nongq'))]:
            
            
            # Load the CSV file into a DataFrame
            df_path = os.path.join(results_dir, model, csv_file)
            df = pd.read_csv(df_path)

            # Add the abstraction level column to the DataFrame
            df['Abs.'] = df['prompt_id'].map(abstraction_levels)

            # Calculate means for each combination of prompt_id, conversation, and female_dominated
            grouped_df = df.groupby(['Abs.', 'model', 'prompt_id', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()
            model = grouped_df['model'].unique()[0]

            # Drop the 'model' column
            model_df = grouped_df.drop(columns=['model'])
                        # Sort by Abs. and prompt_id to ensure correct order
            model_df['prompt_id'] = model_df['prompt_id'].astype(int)
            model_df['Abs.'] = pd.Categorical(model_df['Abs.'], categories=abs_order, ordered=True)
            model_df = model_df.sort_values(by=['Abs.', 'prompt_id'])

            latex_table = r'''
\begin{table*}[ht!]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c}
    \toprule
    & &\multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){3-5} \cmidrule(lr){6-8}
        Abs. & ID & M & F & D & M & F & D \\
        \midrule
'''

            # Sort by prompt_id to ensure order from None to 6
            model_df['prompt_id'] = model_df['prompt_id'].astype(int)
            model_df = model_df.sort_values(by=['Abs.','prompt_id'])

            previous_abs_level = None

            for abs_level, abs_group in model_df.groupby('Abs.'):
                if previous_abs_level is not None:
                    latex_table += "        \\midrule\n"
                for prompt_id, prompt_group in abs_group.groupby('prompt_id'):
                    id_label = 'None' if prompt_id == 0 else str(prompt_id)
                    row_prefix = f"        {abs_level if prompt_id in [0,2,4,6] else ''} & {id_label} & "

                    no_dialogue_fd = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == True)]
                    no_dialogue_md = prompt_group[(prompt_group['conversation'] == False) & (prompt_group['female_dominated'] == False)]


                    row = row_prefix

                    if not no_dialogue_fd.empty:
                        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
                    else:
                        row += " & & & "

                    if not no_dialogue_md.empty:
                        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
                    else:
                        row += " & & "


                    row += r"\\"
                    latex_table += row + "\n"

                if abs_level != '':  # Add averages for all except "None"
                    avg_row_prefix = f"         & \\textbf{{Avg}} & "

                    avg_no_dialogue_fd = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == True)][['male', 'female', 'diverse']].mean(numeric_only=True)
                    avg_no_dialogue_md = abs_group[(abs_group['conversation'] == False) & (abs_group['female_dominated'] == False)][['male', 'female', 'diverse']].mean(numeric_only=True)

                    avg_row = avg_row_prefix

                    if not avg_no_dialogue_fd.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_fd['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_fd['diverse']*100:.1f}\\%}} & "
                    else:
                        avg_row += " & & &"

                    if not avg_no_dialogue_md.empty:
                        avg_row += f"\\textbf{{{avg_no_dialogue_md['male']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['female']*100:.1f}\\%}} & \\textbf{{{avg_no_dialogue_md['diverse']*100:.1f}\\%}}"
                    else:
                        avg_row += " & & "



                    avg_row += r"\\"
                    latex_table += avg_row + "\n"

                previous_abs_level = abs_level

            latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{''' + caption + r'''}
\label{tab:''' + label_suffix + r'''_''' + model_name + r'''}
\end{table*}

'''

            f_out.write(latex_table)
        f_out.write('\\clearpage\n\n')

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/aggregated_results_no_diag.tex


/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/1537791341.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/1537791341.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for abs_level, abs_group in model_df.groupby('Abs.'):
/var/folders/h3/mg_n_0ls7kgb1369wvkmb_dh0000gq/T/ipykernel_48870/1537791341.py:57: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or

In [292]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation

FileNotFoundError: [Errno 2] No such file or directory: '../data/results/alpaca-7b/genderquestion.csv'

In [277]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation
# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){2-7} \cmidrule(lr){8-13}
    Model & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
    & M & F & D & M & F & D & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == True)]
    dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_fd.empty:
        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_md.empty:
        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    if "instruct" in model:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on explicit bias (Task prompt 1).}
\label{tab:explicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'explicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/explicit_default.tex


In [278]:
from itertools import product
# collect all prompt_id = 0
df_0 = []
for model, file_name in product(models, ['non_gq.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['prompt_id'] == 0])
    
df_0 = pd.concat(df_0)

# Calculate means for each combination of prompt_id, conversation
# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c c c c c c}
    \toprule
    & \multicolumn{6}{c}{Without Dialogue} & \multicolumn{6}{c}{With Dialogue} \\
    \cmidrule(lr){2-7} \cmidrule(lr){8-13}
    Model & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7} \cmidrule(lr){8-10} \cmidrule(lr){11-13}
    & M & F & D & M & F & D & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == True)]
    dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == True) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_fd.empty:
        row += f"{dialogue_fd['male'].values[0]*100:.1f}\\% & {dialogue_fd['female'].values[0]*100:.1f}\\% & {dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not dialogue_md.empty:
        row += f"{dialogue_md['male'].values[0]*100:.1f}\\% & {dialogue_md['female'].values[0]*100:.1f}\\% & {dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    if "instruct" in model:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on implicit bias (Average over task prompts 2-4).}
\label{tab:implicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'implicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)

Aggregated LaTeX table saved to ../data/results/implicit_default.tex


In [279]:
from itertools import product
import pandas as pd
import os

# Collect all prompt_id = 0
df_0 = []
models = [
    'llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 
    'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-chat',
    'alpaca-7b'
    
]

model_str_map = {
    'llama3-8b': 'Llama3 8B',
    'llama3-8b-instruct': 'Llama3 8B (Instruct)',
    'mistral-7b': 'Mistral 7B',
    'mistral-7b-instruct': 'Mistral 7B (Instruct)',
    'llama2-7b': 'Llama2 7B',
    'llama2-7b-chat': 'Llama2 7B (Chat)',
    'alpaca-7b': 'Alpaca 7B'
}


for model, file_name in product(models, ['genderquestion.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['prompt_id'] == 0])

df_0 = pd.concat(df_0)

# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c c c c}
    \toprule
    & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    if "instruct" in model:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on explicit bias (task prompt 1).}
\label{tab:explicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'explicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)


FileNotFoundError: [Errno 2] No such file or directory: '../data/results/alpaca-7b/genderquestion.csv'

In [274]:
from itertools import product
import pandas as pd
import os

# Collect all prompt_id = 0
df_0 = []
models = [
    'llama3-8b', 'llama3-8b-instruct', 'mistral-7b', 
    'mistral-7b-instruct', 'llama2-7b', 'llama2-7b-chat'
]

model_str_map = {
    'llama3-8b': 'Llama3 8B',
    'llama3-8b-instruct': 'Llama3 8B (Instruct)',
    'mistral-7b': 'Mistral 7B',
    'mistral-7b-instruct': 'Mistral 7B (Instruct)',
    'llama2-7b': 'Llama2 7B',
    'llama2-7b-chat': 'Llama2 7B (Chat)'
}



for model, file_name in product(models, ['non_gq.csv']):
    # Load the CSV file into a DataFrame
    df_path = os.path.join(results_dir, model, file_name)
    df = pd.read_csv(df_path)
    df_0.append(df[df['prompt_id'] == 0])

df_0 = pd.concat(df_0)

# Calculate means for each combination of model, conversation, and female_dominated
grouped_df_0 = df_0.groupby(['model', 'conversation', 'female_dominated']).mean(numeric_only=True).reset_index()

# Drop the 'prompt_id' column as it is always 0 in this filtered DataFrame
grouped_df_0 = grouped_df_0.drop(columns=['prompt_id'])

# Generate LaTeX table
latex_table = r'''
\begin{table*}[t]
\centering
\small
    % Reduce text size and slightly the gap between columns
    \setlength{\tabcolsep}{4.6pt} % Default: 5pt
    % \resizebox{\textwidth}{!}{  % Alternative method: resize entire table (problem: also resizes line widths)
    \begin{tabular}{l c c c c c c}
    \toprule
    & \multicolumn{3}{c}{Female Dominated} & \multicolumn{3}{c}{Male Dominated} \\
    \cmidrule(lr){2-4} \cmidrule(lr){5-7}
    Model & M & F & D & M & F & D \\
    \midrule
'''

for model in models:
    model_name = model_str_map[model]
    no_dialogue_fd = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == True)]
    no_dialogue_md = grouped_df_0[(grouped_df_0['model'] == model) & (grouped_df_0['conversation'] == False) & (grouped_df_0['female_dominated'] == False)]
    
    row_prefix = f"        {model_name} & "
    row = row_prefix
    
    if not no_dialogue_fd.empty:
        row += f"{no_dialogue_fd['male'].values[0]*100:.1f}\\% & {no_dialogue_fd['female'].values[0]*100:.1f}\\% & {no_dialogue_fd['diverse'].values[0]*100:.1f}\\% & "
    else:
        row += " & & & "
    
    if not no_dialogue_md.empty:
        row += f"{no_dialogue_md['male'].values[0]*100:.1f}\\% & {no_dialogue_md['female'].values[0]*100:.1f}\\% & {no_dialogue_md['diverse'].values[0]*100:.1f}\\% "
    else:
        row += " & & "
    
    row += r"\\"
    if "instruct" in model:
        latex_table += row + "\n        \\midrule\n"
    else:
        latex_table += row + "\n        \n"

latex_table += r'''
\bottomrule
\end{tabular}
% } % end \resizebox
\caption{Results for all models on implicit bias (average over task prompts 2-4).}
\label{tab:implicit_default}
\end{table*}
'''

# Save the LaTeX table to a file
output_tex_file = os.path.join(results_dir, 'implicit_default.tex')
with open(output_tex_file, 'w') as f_out:
    f_out.write(latex_table)

print("Aggregated LaTeX table saved to", output_tex_file)


Aggregated LaTeX table saved to ../data/results/implicit_default.tex
